# Context Managers

Some objects need to clean up resources when done. Examples:
* `File` object needs to `close()` call
* A network connection may need to close
* A data-intensive operation may need to `del` the data

We may think about that in a context of following pattern:

```python
r = acquire_resource()
try:
    do_something(r)
finally:
    release_resource(r)
```

`__with__` provides a block that "cleans up" when exited. It can handle exceptions that occur within the block and also execute code when entered.


```python
with acquire_resource() as r:
    do_something(r)
```

## Context Manager Protocol

In [1]:
class MyClass:
    def __enter__(self):
        print('Entered "with" :)')
        return self
        
    def __exit__(self, type, value, traceback):
        print(f'Error type: {type}')
        print(f'Error value: {value}')
        print(f'Error traceback: {traceback}')
        
    def hi(self):
        print(f'Hi from {id(self)} instance!')
        
with MyClass() as mobj:
    mobj.hi()

Entered "with" :)
Hi from 4535246480 instance!
Error type: None
Error value: None
Error traceback: None


In [2]:
with MyClass() as mobj:
    mobj.hi()
    1/0

Entered "with" :)
Hi from 4535244880 instance!
Error type: <class 'ZeroDivisionError'>
Error value: division by zero
Error traceback: <traceback object at 0x10e52ba50>


ZeroDivisionError: division by zero

The steps which are taken by the `with` statement when an error is encountered:

1. It passes the type, value and traceback of the error to the `__exit__` method
2. It allows the `__exit__` method to handle the exception
3. If `__exit__` returns `True` then the exception will be gracefully handled
4. If anything other than `True` is returned by the `__exit__` method then the exception will be raised by the `with` statement


Conceptually everything works like that:

```python
manager = acquire_resource()
r = manager.__enter__()
try:
    do_something(r)
finally:
    exc_type, exc_value, tb = sys.exc_info()
    suppress = manager.__exit__(exc_type, exc_value, tb)
    if exc_value is not None and not suppress:
        raise exc_value
```

## Multiple context managers

`with` statement supports multiple arguments:

```python
with open('data.txt', 'r') as source, \
     open('data_copy.txt', 'w') as destination:
    destination.write(source.read())
```

This code is equivalent to:

```python
with open('data.txt', 'r') as source:
    with open('data_copy.txt', 'w') as destination:
        destination.write(source.read())
```

_Therefore there is no need to write nested context managers._

P.S. In the case of multiple (nested) context managers `__exit__` is called in reverse order.

## Example: opened

⚠️ Just for educational purposes. Files in Python already support the context manager protocol.

In [3]:
from functools import partial

In [4]:
class opened:
    def __init__(self, path, *args, **kwargs):
        self.opener = partial(open, path, *args, **kwargs)
        
    def __enter__(self):
        self.handle = self.opener()
        return self.handle
    
    def __exit__(self, *exc_info):
        self.handle.close()
        del self.handle

In [5]:
with opened("tmp.txt", mode="rt") as handle:
    pass

## [tempfile — Generate temporary files and directories](https://docs.python.org/3/library/tempfile.html#module-tempfile)

In [6]:
import tempfile

In [7]:
with tempfile.TemporaryFile() as handle:
    path = handle.name
    print(path)

58


In [8]:
open(path)

OSError: [Errno 9] Bad file descriptor

## Example: syncronized

⚠️ Just for educational purposes.

In [9]:
import threading

class synchronized:
    def __init__(self):
        self.lock = threading.Lock()
        
    def __enter__(self):
        self.lock.acquire()
        
    def __exit__(self, *exc_info):
        self.lock.release()

In [10]:
with synchronized():
    pass

## Example: cd

In [11]:
import os

class cd:
    def __init__(self, path):
        self.path = path
        
    def __enter__(self):
        self.saved_cwd = os.getcwd()
        os.chdir(self.path)
        
    def __exit__(self, *exc_info):
        os.chdir(self.saved_cwd)

In [12]:
print(os.getcwd())

/Users/akrisanov/Development/python_notebook


In [13]:
with cd("/tmp"):
    print(os.getcwd())

/private/tmp


## [contextlib — Context Manager Utilities](https://pymotw.com/3/contextlib/)

### contextlib.closing

Return a context manager that closes thing upon completion of the block.

In [14]:
from contextlib import closing
from urllib.request import urlopen

url = "https://ya.ru"
with closing(urlopen(url)) as page:
    pass

### contextlib.redirect_stdout

Redirecting Output Streams.

In [15]:
from contextlib import redirect_stdout
import io

In [16]:
handle = io.StringIO()
with redirect_stdout(handle):
    print("Hello, World!")

In [17]:
handle.getvalue()

'Hello, World!\n'

### contextlib.suppress

In [18]:
from contextlib import suppress
with suppress(FileNotFoundError):
    os.remove("abcd.txt")

Context manager implementation:

In [19]:
class suppress:
    def __init__(self, *suppressed):
        self.suppressed = suppressed
        
    def __enter__(self):
        pass
    
    def __exit__(self, exc_type, exc_value, tb):
        return (exc_type is not None and issubclass(exc_type, suppressed))

### contextlib.ContextDecorator

The class `ContextDecorator` adds support to regular context manager classes to let them be used as function decorators as well as context managers.

Example:

```python
def f():
    with context():
        # ...
        
        
# vs

@context
def f():
    # ...
```

### @contextlib.contextmanager

The `contextmanager` decorator should be used on a generator function. The `__enter__` and `__exit__` methods will be dynamically implemented based on the code that wraps the `yield` statement of the generator.

In [20]:
import contextlib

@contextlib.contextmanager
def my_context():
    print('do something first')
    yield
    print('do something else')

In [21]:
with my_context():
    print('Hi!')

do something first
Hi!
do something else


### yielding a value to the caller

In [22]:
import contextlib

@contextlib.contextmanager
def my_context():
    print('do something first')
    try:
        yield 42
    finally:
        print('do something else')
    
with my_context() as value:
    print(value)

do something first
42
do something else


### contextlib.ExitStack

In [23]:
from contextlib import ExitStack

In [24]:
def merge_logs(output_path, *logs):
    with ExitStack() as stack:
        handles = [stack.enter_context(open(log)) for log in logs]
        output = open(output_path, "wt")
        stack.enter_context(output)
        merge(output, handles)